# --------------------------------------------------------------------------------------

# Algoritmo - Determinação das velocidades GPS
### _Autor: Antônio Claudio Dutra Batista_
### _Orientador: Francisco Moraes de Oliveira Neto_
### _Metodologia: Velocidades médias entre os registros do GPS_
''' As informações dos veículos no trecho foram obtidos pela integração da base da bilhegem a base do gps pelo criteiro de tempo mais próximo (Não foi localizado as validações em _TERMINAIS e em BRT´s_ , pois é de interesse saber as informações dos veculos das linhas filtradas e com seus veiculos em operação!) '''

# 1.0  Leitura das Bases

In [ ]:
# Importando bibliotecas e funções necessárias 
import folium 
import numpy as np 
import pandas as pd
import geopandas as gpd
from shapely import Point
import plotly.graph_objs as go
from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster

## 1.1.0 leitura do base - GPS (Tratamento INICIAL)

In [ ]:
# Definindo o número de linhas que você deseja ler
numero_de_linhas_para_ler = 45000000  # Substitua pelo número desejado
dtype_specification = {'longitude': 'float32', 'latitude': 'float32', 'vehicle_vehicleid': 'int32'}

# Usando o pandas para ler o CSV, limitando o número de linhas com a opção nrows
mes_ate_45_milhoes = pd.read_csv('Paint112018.csv',usecols=['latitude', 'longitude', 'metrictimestamp', 'vehicle_vehicleid'], dtype=dtype_specification, nrows=numero_de_linhas_para_ler)

In [ ]:
# Lendo restante do arquivo
mes_resto = pd.read_csv('Paint112018.csv', usecols=['latitude', 'longitude','metrictimestamp', 'vehicle_vehicleid'],dtype=dtype_specification, skiprows=range(1, numero_de_linhas_para_ler + 1))

In [ ]:
# Juntando em um mesmo df
GPS_I_H =  pd.concat([mes_ate_45_milhoes, mes_resto]) 

In [ ]:
# Informações da base
GPS_I_H.info()

In [ ]:
# Formatando coluna com informaçãoes de data
GPS_I_H['momento'] = pd.to_datetime(GPS_I_H['metrictimestamp'],format= '%Y%m%d%H%M%S')

In [ ]:
# Removendo coluna desnecessária 
GPS_I_H.drop("metrictimestamp", axis= 1, inplace= True)

In [ ]:
# Renomear coluna de vehicle_vehicleid para id_veiculo:
GPS_I_H.columns=['latitude', 'longitude','id_veiculo', 'momento']

In [ ]:
GPS_I_H.sort_values("momento", inplace = True)

In [ ]:
GPS_I_H

In [1]:
''' A coluna momento terá o mesmo nome da coluna de tempo da bilhetagem e apos a mesclagem para obter informaçõs dos veiculos associando as suas linhas de operação somente ira permanecer a coluna de monento da bilhetagem '''

' A coluna momento terá o mesmo nome da coluna de tempo da bilhetagem e apos a mesclagem para obter informaçõs dos veiculos associando as suas linhas de operação somente ira permanecer a coluna de monento da bilhetagem '

## 1.1.1 leitura da base - Bilhetagem (Tratamento Inicial)

In [ ]:
# Arquivos são do mês de novembro de 2018.
# Definindo tipos de variáveis com tamanhos menores
tipos_specification = {'id': 'int32', 'linha': 'int32'}

# Lista para armazenar os dataframes de cada grupo de 10 dias
grupos_dias = []

# Loop para ler os arquivos CSV e concatená-los em grupos de 10 dias
for i in range(1, 31, 10):  # Início, fim (não inclusivo), passo
    dias = [pd.read_csv(f'2018-11-{str(j).zfill(2)}.csv', usecols=['id', 'linha', 'nome_linha', 'prefixo_carro',
    'nome_cartao', 'sentido_viagem', 'dia', 'momento'], dtype=tipos_specification, sep=',') for j in range(i, i + 10)]
    grupo = pd.concat(dias)
    grupos_dias.append(grupo)

In [ ]:
# Concatenando os grupos de 10 dias
mes_tudo = pd.concat(grupos_dias)

In [ ]:
mes_tudo

## 1.1.3 Tratamento da base para  associação com o gps

In [ ]:
# informações do df
mes_tudo.info()

In [ ]:
# Mostrando dados nulos presentes
mes_tudo.isnull().sum()

In [ ]:
# Juntando dados daa colunas de interesse (dia, momento):
mes_tudo["momento"] = mes_tudo['dia'].astype(str) + ' ' + mes_tudo['momento'].astype(str)

In [ ]:
# REMOVENDO COLUNAS DESNECESSÁRIAS!
mes1 = mes_tudo.drop(['dia'], axis=1)

In [ ]:
mes1["momento"] = pd.to_datetime(mes1["momento"])

In [ ]:
# Visualizando dados nulos
mes1[mes1['prefixo_carro'].isnull()]

In [2]:
''' Exitem validações de usuario na base que não tiveram a identificação do veiculo registrada - por serem uma pequena parte dos dados serão removidos para proceguimento do algoritimo '''

' Exitem validações de usuario na base que não tiveram a identificação do veiculo registrada - por serem uma pequena parte dos dados serão removidos para proceguimento do algoritimo '

In [ ]:
# Removendo dados nulos na coluna prefixo_carro
mes_ = mes1.dropna(subset = ['prefixo_carro'])

In [ ]:
# Função para identificar se há caracters
def tem_caracters(col):
    return any(char.isalpha() for char in str(col))

In [ ]:
# Verificando 
prefixo_CARACTERS = pd.DataFrame(mes_["prefixo_carro"])
prefixo_resultado = prefixo_CARACTERS.applymap(tem_caracters)
prefixo_resultado[prefixo_resultado.prefixo_carro == True]

In [3]:
''' Existem alguns dias que a coluna de identificação dos veiculos apresentam caracteres na sua composiçao, por serem tambem um pequena parcela serão removidos para aplicação do metodo, mas inspeções poderiam explicar mais a ocorrencias dessa questão como de outras ja citadas '''

' Existem alguns dias que a coluna de identificação dos veiculos apresentam caracteres na sua composiçao, por serem tambem um pequena parcela serão removidos para aplicação do metodo, mas inspeções poderiam explicar mais a ocorrencias dessa questão como de outras ja citadas '

In [ ]:
# Removenfdo identificação de veiculos com caracteris da coluna de prefixo_carro
mes = mes_[pd.to_numeric(mes_['prefixo_carro'], errors='coerce').notnull()]

In [ ]:
# Trasnformando coluna de prefixo carro em inteiro
mes['prefixo_carro'] = mes['prefixo_carro'].astype('int32')

In [ ]:
mes.info()

In [ ]:
# Última versão do df MES adequado para localização
mes

## 1.3 leitura da base - Dicionário

In [ ]:
# df', abaixo:
d1 = pd.read_csv('veiculos2018.csv', sep= ';')

In [ ]:
# Mostrando Df
display(d1)

In [ ]:
d1

In [4]:
''' Serão removidos os dados de caracteres na coluna de identificação dos veiculos, mas isso pode ser o principal motivo do não georreferenciamneto da base completa da bilhetagem '''

' Serão removidos os dados de caracteres na coluna de identificação dos veiculos, mas isso pode ser o principal motivo do não georreferenciamneto da base completa da bilhetagem '

In [ ]:
# Removenfdo identificação de veiculos com caracteris da coluna de cod_veiculo
d01 = d1[pd.to_numeric(d1['cod_veiculo'], errors='coerce').notnull()]

In [ ]:
d01.info()

In [ ]:
# Trasnformando coluna de prefixo carro em inteiro
d01['cod_veiculo'] = d01['cod_veiculo'].astype('int32')
d01['id_veiculo'] = d01['id_veiculo'].astype('int32')

In [ ]:
d01['cod_veiculo'].sort_index()

In [ ]:
# Contagem de dados nulos 
d01.isnull().sum()

In [ ]:
d01

# 2.0 Junção da bilhetagem com o dicionário 

In [ ]:
# Acrescentando dicionário
mes_d1= mes.merge(d01, left_on= 'prefixo_carro', right_on='cod_veiculo')

In [ ]:
# Base da Bilhetagem adicionado o dicionario
mes_d1

In [ ]:
# informações da base 
mes_d1.info()

In [ ]:
# Colocando momento em ordem para operação de mesclagem
mes_geo = mes_d1.sort_values('momento')

In [ ]:
# Reiniciando a contagem dos index
mes_geo.reset_index(inplace=True)
# Reiniciando a contagem dos index
mes_geo.drop(columns='index', inplace=True)

In [ ]:
mes_geo

In [ ]:
# REMOVENDO COLUNA DESNECESSÁRIA - informação de identificação dos veiculos agora presente em duas colunas!
mes_georr = mes_geo.drop(["prefixo_carro"], axis=1)

In [ ]:
mes_georr.info()

In [5]:
''' AGORA TEMOS DATAFRAME DA BILHETAGEM E DO GPS NO FORMATO QUE PODEMOS FAZER A associação pelo momento para o mesmo veiculo '''

' AGORA TEMOS DATAFRAME DA BILHETAGEM E DO GPS NO FORMATO QUE PODEMOS FAZER A associação pelo momento para o mesmo veiculo '

# 3.0 Integração da coordenada aproximada do ônibus na bilhetagem

In [ ]:
# Coordenada anterior mais proxima do ônibus é extraída da base de GPS e integrada na base da bilhetagem (SOMENTE COLUMAS MOMENTO E ID)
mes_d1_GPS = pd.merge_asof(mes_georr, GPS_I_H, left_on='momento', right_on='momento', by='id_veiculo', direction ='nearest')

In [ ]:
mes_d1_GPS.isnull().sum()

In [6]:
''' Alguns dados geometricos possuem valores nulos - esses valores ja vem com esse configuração da base do GPS '''

' Alguns dados geometricos possuem valores nulos - esses valores ja vem com esse configuração da base do GPS '

In [ ]:
# Base da bilhetagem localizada 
mes_d1_GPS

# 4.0 Separando dia para geração e analises dos perfis veiculares

##  4.1 Separando dia da bilhetagem localizada para identificação dos veiculos das linhas 

In [ ]:
# FILTRANDO VALIDAÇÕES NO DIA ESPECIFICO
validações_trecho_dia = mes_d1_GPS[(mes_d1_GPS.momento<='2018-11-01 23:59:59')]
   # & (mes_d1_GPS.momento<=-38.56944)]

In [ ]:
validações_trecho_dia

##  4.2 Separando dia do GPS (registros do dia desejado)

In [ ]:
# FILTRANDO REGISTROS NO DIA ESPECIFICO
dia_especifico_gps = GPS_I_H[(GPS_I_H.momento<='2018-11-01 23:59:59')]
   # & (mes_d1_GPS.momento<=-38.56944)]

In [ ]:
dia_especifico_gps

# 5.0 Extraindo velocidades veiculares a partir do GPS
''' Esse algoritimo é aplicado por dia que se deseja fazer a analise '''

## 5.1 Identificando linhas operando no trecho e  horario de interesse
''' OBS: É NECESSÁRIO DEFINIR O HORARIO PARA GERAÇÃO E ANALISES DOS PERFIS '''

In [7]:
''' SERÁ IDENTIFICADAS AS LINHAS NESSA REGIÃO, POIS NÃO TEM NUMHUMA RUA OU AVENIDA QUE CORTA O TRECHO DE INTERESSE NA TRANSVERSAL E ASSIM POSSIVELMENTE NÃO TEM OUTRA LINHAS NESSES SENTIDOS: -3.734498870509772, -38.55723488359994; -3.733084073193523, -38.55346122624892 '''

' SERÁ IDENTIFICADAS AS LINHAS NESSA REGIÃO, POIS NÃO TEM NUMHUMA RUA OU AVENIDA QUE CORTA O TRECHO DE INTERESSE NA TRANSVERSAL E ASSIM POSSIVELMENTE NÃO TEM OUTRA LINHAS NESSES SENTIDOS: -3.734498870509772, -38.55723488359994; -3.733084073193523, -38.55346122624892 '

In [ ]:
# FILTRANDO VALIDAÇÕES EM TRECHO ESPECIFICO 
validações_trecho = validações_trecho_dia[(validações_trecho_dia.latitude>=-3.734498870509772)
    & (validações_trecho_dia.longitude>=-38.55723488359994) & (validações_trecho_dia.longitude<=-38.55346122624892)
                                & (validações_trecho_dia.latitude<=-3.733084073193523)]

In [8]:
''' OBS: É INTERESSANTE FILTAR O HORARIO PARA GERAÇÃO DE PERFIS EM INTERVALOS CURTOS, PARA QUE NÃO SE TENHA SOBREPOSIÇÃO DE REGISTROS DE UM MESMO VEICULO NO MESMO HORARIO '''

' OBS: É INTERESSANTE FILTAR O HORARIO PARA GERAÇÃO DE PERFIS EM INTERVALOS CURTOS, PARA QUE NÃO SE TENHA SOBREPOSIÇÃO DE REGISTROS DE UM MESMO VEICULO NO MESMO HORARIO '

In [ ]:
# Filtragem no intervalo de interesse
validações_linhas_hora = validações_trecho[(validações_trecho.momento>= '2018-11-01 17:00:00') & 
                                (validações_trecho.momento<='2018-11-01 18:00:00')]

In [ ]:
# LINHAS NA REGIÃO E NO HORARIO  
linhas = validações_linhas_hora['linha'].unique()
linhas

In [9]:
''' OU SEJA, A LINHA COM MAIS VALIDAÇÕES NO HORARIO DE INTERESSE É AQUELA QUE PODE SER CONSIDERADA, POIS SE TEM VALIDAÇÕES NESSA LINHA, SIGUINIFICA QUE ELA POSSUE VEICULOS EM OPERAÇÃO PARA ESSA DEMANDA! '''

' OU SEJA, A LINHA COM MAIS VALIDAÇÕES NO HORARIO DE INTERESSE É AQUELA QUE PODE SER CONSIDERADA, POIS SE TEM VALIDAÇÕES NESSA LINHA, SIGUINIFICA QUE ELA POSSUE VEICULOS EM OPERAÇÃO PARA ESSA DEMANDA! '

## 5.2 Identificação de veiculos da linhas de interesse

In [ ]:
# Verificando os diferentes veiculos da linha de interesse
veiculos_linha_especifica = validações_linhas_hora['id_veiculo'].unique()
veiculos_linha_especifica

## 5.3  _Filtragem de veiculos da linhas de interesse_
  ''' Filtrando registros na base do GPS agora com os veiculos ja identificados '''

In [ ]:
###############################################  Filtrando veiculos espeficos  ###########################################################
lista_veiculos = veiculos_linha_especifica

viculos_especificos__ = dia_especifico_gps[dia_especifico_gps["id_veiculo"].isin(lista_veiculos)]
viculos_especificos__
##########################################################################################################################################

In [ ]:
# FILTRANDO DADOS DO VECIULO EM TRECHO DE INTERESSE
viculos_especificos__ = viculos_especificos__[(viculos_especificos__.latitude>=-3.73667)
    & (viculos_especificos__.longitude>=-38.56944) & (viculos_especificos__.longitude<=-38.54167)
                                & (viculos_especificos__.latitude<=-3.73083)]

In [10]:
''' Foi atribuido um trecho para captação dos registro do GPS antes e depois do trecho real de interesse: Comeco ((-3.73667, -38.56944)) e Fim (-3.73083, -38.54167) ''' 

' Foi atribuido um trecho para captação dos registro do GPS antes e depois do trecho real de interesse: Comeco ((-3.73667, -38.56944)) e Fim (-3.73083, -38.54167) '

In [ ]:
# Filtrando os registros no horario de pico do dia a ser extraido as velocidades
viculos_especificos_TRECHO__ = viculos_especificos__[(viculos_especificos__.momento>=
                '2018-11-01 18:00:00') & (viculos_especificos__.momento<='2018-11-01 19:00:00')]

In [ ]:
# Colocando coluna de momento em ordem
viculos_especificos_TRECHO_= viculos_especificos_TRECHO__.sort_values('momento')

In [ ]:
viculos_especificos_TRECHO_

In [ ]:
# Criando coluna geometrica
viculos_especificos_TRECHO_['geometry'] = viculos_especificos_TRECHO_.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)

In [ ]:
viculos_especificos_TRECHO_

## 5.4 Vizualização dos dados no trecho
'' OBS: QUANDO SE CLICA NOS PONTOS NO MAPA É POSSIVEL VER OS REGISTROS E QUAIS VEICULOS ESTÃO RELAMENTE NA DIREÇÃO DO TRECHO DE ESTUDO, ASSIM SERÃO REMIVIDOS AQUELES QUE NÃO OBEDECEREM ESSA CONDIÇÃO '''

In [ ]:
# Criando media para visulização do mapa proximo a região dos dados geometricos
lon = viculos_especificos_TRECHO_['longitude'].mean()
lat = viculos_especificos_TRECHO_['latitude'].mean()

In [ ]:
# Criando um mapa centrado em FORTALEZA
mapa_viculo_especifico_TRECHO = folium.Map(location=[lat, lon], zoom_start=5)

def popup_content(id_veiculo, id_veiculo_clicado):
    cor_popup = 'blue' if id_veiculo != id_veiculo_clicado else 'red'
    return f'<div style="color: {cor_popup};">{id_veiculo}</div>'

for index, row in viculos_especificos_TRECHO_.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  icon=folium.Icon(icon='glyphicon-pushpin', color='blue'),
                  popup=folium.Popup(popup_content(row['id_veiculo'], ''), max_width=300)).add_to(mapa_viculo_especifico_TRECHO)

In [ ]:
mapa_viculo_especifico_TRECHO

In [11]:
''' A PARTIR DESSA ANALISE É POSSIVEL VERIFICAR QUAIS VEICULOS NÃO ESTÃO NA ROTA DE ESTUDO '''

' A PARTIR DESSA ANALISE É POSSIVEL VERIFICAR QUAIS VEICULOS NÃO ESTÃO NA ROTA DE ESTUDO '

In [ ]:
# Removendo dados do veiculo (os) não necessários
viculos_especificos_TRECHO_ = viculos_especificos_TRECHO_[viculos_especificos_TRECHO_['id_veiculo'] != 33095]  
# ESPECIFICAR CASO TENHA ALGUM VEICULO FORA DA ROTA

## _5.5 Criando lista com um df para cada veiculo das linhas filtradas_

In [ ]:
# Criando função para pegar os dados de um mesmo veiculo e separar em diferentes dfs
def agrupar_por_ids(viculos_especificos_TRECHO_, id_veiculo):
    # Agupando dados por id
    grupos = viculos_especificos_TRECHO_.groupby('id_veiculo')
    
    # Lista para guardar os GeoDataFrames
    geodataframes = []
    
    # Iterando sobre os grupos 
    for _, grupo in grupos:
        # Criando GeoDataFrame a partir do DataFrame do grupo
        geo_df = gpd.GeoDataFrame(grupo, geometry=gpd.points_from_xy(grupo.longitude, grupo.latitude))
        geodataframes.append(geo_df)
        
    geo_data_ordem = geodataframes
    
    return geo_data_ordem

In [ ]:
# Aplicando a função
viculos_especificos_TRECHO__n = agrupar_por_ids(viculos_especificos_TRECHO_, 'id_veiculo')

In [ ]:
# Visualizando df especifico 
viculos_especificos_TRECHO__n[0]

## 5.6 Extraindo velocidades instantâneas para veiculos especificos

In [ ]:
# Colocando coluna de momento de cada df das lista
# Função para processar um DataFrame
def ordem_df(df):
    # Ordenar o DataFrame pela coluna 'momento'
    df = df.sort_values('momento')
    
    # Restaurando o índice do DataFrame após a ordenação
    df = df.reset_index(drop=True)    
    return df

# Aplicando a função para cada DataFrame na lista
viculos_especificos_TRECHO__ordem = [ordem_df(df) for df in viculos_especificos_TRECHO__n]

In [ ]:
# Criando função para criar coluna de gemetrica anterior em cada DataFrame
def processar_dataframe_dist(df):
    df['geometry_anterior'] = df['geometry'].shift(1)
    return df

In [ ]:
# Aplicando a função para cada DataFrame na lista
viculos_especificos_TRECHO_geo_ant = [processar_dataframe_dist(df) for df in viculos_especificos_TRECHO__ordem]

In [ ]:
# Acesso ao resultado, por exemplo, para o primeiro DataFrame na lista processada
primeiro_dataframe_processado = viculos_especificos_TRECHO_geo_ant[0]
primeiro_dataframe_processado

In [ ]:
# Criando função para criar coluna de momento anterior em cada DataFrame
def processar_dataframe_momento(df):
    df['momento_anterior'] = df['momento'].shift(1)
    return df

# Aplicando a função para cada DataFrame na lista
viculos_especificos_TRECHO_geo_mom_ant = [processar_dataframe_momento(df) for df in viculos_especificos_TRECHO_geo_ant]

In [ ]:
# Acessando a resultado, por exemplo, para o primeiro DataFrame na lista processada
primeiro_dataframe_processado = viculos_especificos_TRECHO_geo_mom_ant[1]
primeiro_dataframe_processado

In [ ]:
# instalando bibloteca para trazer metodo de calcular distancia 
!pip install geopy
from geopy.distance import geodesic  

# Função para calcular distância entre pontos
def calcular_distancia(row):
    if row['geometry'] is None or row['geometry_anterior'] is None:
        return None
    return geodesic((row.geometry.x, row.geometry.y), (row.geometry_anterior.x, row.geometry_anterior.y)).meters

# Função para processar um DataFrame
def processar_distancia(df):
    df['distancia'] = df.apply(calcular_distancia, axis=1)
    return df

In [ ]:
# Aplicando a função para cada DataFrame na lista
viculos_especificos_TRECHO_dist_ = [processar_distancia(df) for df in viculos_especificos_TRECHO_geo_mom_ant]

In [ ]:
viculos_especificos_TRECHO_dist_[0]

In [ ]:
# Função para calcular a diferença de tempo
def calcular_diferenca_tempo(row):
    if pd.isnull(row['momento']) or pd.isnull(row['momento_anterior']):
        return None
    return (row['momento'] - row['momento_anterior']).total_seconds()

# Função para processar um GeoDataFrame
def processar_geodataframe_temp(gdf):
    # Aplicar a função calcular_diferenca_tempo à coluna 'momento' e criar nova coluna 'deltaT'
    gdf['deltaT'] = gdf.apply(calcular_diferenca_tempo, axis=1)
    
    # Converter a coluna 'deltaT' para o tipo de dados float
    gdf['deltaT'] = gdf['deltaT'].astype('float')
    
    # Adicionar outras operações ou transformações conforme necessário
    
    return gdf

In [ ]:
# Aplicando função 
viculos_especificos_TRECHO_dist_temp = [processar_geodataframe_temp(gdf) for gdf in viculos_especificos_TRECHO_dist_]

In [ ]:
# Função para calcular a velocidade instantânea
def calcular_velocidade_instantanea(row):
    if pd.isnull(row['distancia']) or pd.isnull(row['deltaT']) or row['deltaT'] == 0:
        return None
    return row['distancia'] / row['deltaT']

# Função para processar um GeoDataFrame
def processar_geodataframe_velocidade(gdf):
    # Calcular a velocidade instantânea
    gdf['velocidade'] = gdf.apply(calcular_velocidade_instantanea, axis=1)
    
    return gdf

In [ ]:
#########################_________Aplicando a função para cada GeoDataFrame na lista processada__________#######################
viculos_especificos_TRECHO_velocidade_tudo = [processar_geodataframe_velocidade(gdf) for gdf in 
                                              viculos_especificos_TRECHO_dist_temp]
################################################################################################################################

In [ ]:
viculos_especificos_TRECHO_velocidade_tudo[0]

In [ ]:
# Dropando colunas desnecessárias 
# Lista de colunas a serem removidas
colunas_desnecessarias = ['geometry_anterior', 'momento_anterior', 'distancia']

# Função para remover colunas desnecessárias
def remover_colunas_desnecessarias(gdf):
    # Remover as colunas desnecessárias
    gdf = gdf.drop(columns=colunas_desnecessarias, errors='ignore')
    
    # Adicionar outras operações ou transformações conforme necessário
    
    return gdf

In [ ]:
# Aplicando a função para cada GeoDataFrame na lista de velocidades
viculos_especificos_TRECHO_velocidades = [remover_colunas_desnecessarias(gdf) for gdf in viculos_especificos_TRECHO_velocidade_tudo]

In [ ]:
# Versão final da lista com df para todos os veiculos da lista (filtrando df2, que é o elemento 1 da lista!)
viculos_especificos_TRECHO_velocidades[0]

## 5.6.1 Adicioando dados da primeira parada 
''' Utilizando como ponto de referencia a parada inicial para perfis de velocidades de veiculos no sentido da primeira estação centro, e como referencia a ultima estação no sentido contrario '''

In [ ]:
# Trazendo base com as paradas de interesse
paradas_interesse = pd.read_excel('paradas_interesse_tudo_com_distancias.xlsx')

In [ ]:
paradas_interesse

In [ ]:
# Tramanho do trecho
display(paradas_interesse['distancia'].sum())

In [ ]:
# CRIANDO DF COM OS DADOS GEOMETRICOS DA 1ª PARADA E DA ULTIMA PARA DISTANCIA EM RELAÇÃO A UMA OU OUTRA
# USA-SE OS DADOS GEOMETRICOS DA ULTIMA PARADA QUANDO QUEREMOS OS VEICULOS NA VOLTA! 
data_ = ['2018-11-01 00:00:00']
data = pd.to_datetime(data_)
df_paradadas_  = {'latitude': [-3.736013],
                  'longitude': [-38.566300],
                  'id_veiculo': [np.nan],
                  'momento': [np.nan],
                  'geometry': [np.nan],
                  'deltaT': [np.nan],
                  'velocidade':  [np.nan],
                
        }

df_paradadas__ = pd.DataFrame(df_paradadas_)
df_paradadas__['momento'] = data
df_paradadas__

In [ ]:
# Info dos dados 
df_paradadas__.info()

In [ ]:
''' CRIANDO MESMO DF COM INFROMAÇÕES GEOMETRICAS DA ULTIMA PARADA '''
df_paradadas_fim__ = df_paradadas__
df_paradadas_fim__['latitude'] = -3.731264
df_paradadas_fim__['longitude'] = -38.543328

In [ ]:
# Criando a informação da geometria no df
df_paradadas__['geometry'] = df_paradadas__.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)

In [ ]:
 # Convertendo a coluna 'nova_geometria' para a coluna de geometria principal
df_paradada_inicio = gpd.GeoDataFrame(df_paradadas__, geometry='geometry')
df_paradada_fim = gpd.GeoDataFrame(df_paradadas_fim__, geometry='geometry')

## 5.7 Filtrando dados no seguimento para cada df da lista

In [ ]:
# FILTRANDO DADOS PARA SEGUIMENTO ENTRE A PRIMERA E ULTIMA PARADA DE INTERESSE
def filtrar_por_seguimento_(gdf):
    # Aplicar o filtro
    filtro = (gdf['longitude'] >= -38.566300) & (gdf['longitude'] <= -38.543328)
    gdf_filtrado = gdf[filtro]
    
    return gdf_filtrado

# Aplicando a função para cada GeoDataFrame na lista sem colunas desnecessárias
resultados = [filtrar_por_seguimento_(gdf) for gdf in viculos_especificos_TRECHO_velocidades]

In [ ]:
# Acessando resultado para o segundo dataframe que é o elemento com indice 01
resultados[0]

## 5.7.1 Adequando colunas para padronização das distancias dos perfis
*  SE DEVE ADICIONAR OS DADOS DA ESTAÇÃO PARA PEGAR OS VEICULOS EM OPERAÇÃO NO SENTIDO A PARTIR DELA;
* EXEMPLO: VEICULOS VOLTA SE DEVE ADICOANR DADSOS DA PARADA FIM!

In [ ]:
#  ADICIONANDO DADOS DA ESTAÇÃO PARA CALCULOS DE DISTANCIAS 
from pandas import concat  

# Concatenando os dataframes, adicionando a nova coluna
for i, gpf in enumerate(resultados):
    resultados[i]  = gpf.append(df_paradada_inicio, ignore_index=True)

In [ ]:
# COLOCANDO DADOS DA COLUNA MOMENOTO EM ORDEM 
for i, gpf in enumerate(resultados):
    resultados[i] = gpf.sort_values(by='momento')

In [ ]:
resultados[1]

## 5.7.2 Realizando medição das distancia em relação a parada de interesse

In [ ]:
# Crindo coluna de geometry anterior com função de calculo de distancia ja criada
resultados_ordem_dist = [processar_dataframe_dist(df) for df in resultados]

In [ ]:
resultados_ordem_dist[0]

In [ ]:
# Aplicando função de distancia também feita anteriormente
resultados_dist_tudo = [processar_distancia(df) for df in resultados_ordem_dist]

In [ ]:
resultados_dist_tudo[4]

In [ ]:
# Criando coluna com distancias acumuladas (Em relação a parada de interesse)
def calcular_distancia_acumulada(resultados, coluna_distancia):
    resultados_atualizados = []
    # Iterando sobre cada GeoDataFrame na lista 'resultados'
    for gdf in resultados_dist_tudo:
        # Calculando a distância acumulada
        gdf['distancia_acumulada'] = gdf[coluna_distancia].cumsum()

        # Adicionando o GeoDataFrame atualizado à lista
        resultados_atualizados.append(gdf)

    return resultados_atualizados

In [ ]:
# Chamando a função para calcular a distância acumulada
resultados = calcular_distancia_acumulada(resultados_dist_tudo, 'distancia')

In [ ]:
# LISTA COM DATAFRAMES COM DADOS DE IDA E VOLTA 
len(resultados)

## _5.7.3 Informações dos df´s gerados com velocidades_

In [ ]:
# INFORMAÇÕES DE CADA DF DA LISTA COM VELOCIDADES
for dfs in resultados:
    display(dfs.info())

In [ ]:
# EXIBINDO CADA DF DA LISTA RESULTADOS COM VELOCIDADES
for dfs in resultados:
    display(pd.DataFrame(dfs))

# 6.0 Identificação perfis no mesmo sentido
    ''' INICIALMENTE ADICIONAMOS NOS DADOS DAS VELOCIDADES A LOCALIZAÇÃO DAS ESTAÇÕES DE INTERESSE PARA MEDIR AS DISTANCIAS DOS REGISTOS EM RELAÇÃO A ELA, ENTÃO BASTA FILTRAR AQUELES PERFIS QUE OBEDECEM A LOGICA DA DISTANCIA ACUMULADA SER PEQUENA E IR CRESCENDO COM O PASSAR DO TEMPO, ESSE SENDO UM INDICATIVO DE QUE ESSE VEICULO ESTÁ NO SENTIDO DE INTERESSE ''' 

In [ ]:
# Comprimento do Trecho em analise
comprimento_trecho = 2583.53007784341
print(comprimento_trecho)

In [ ]:
# FILTRANDO SOMENTE UMA VIAGEM SEGUIDA NO SEU INTEVALO (TRECHO) DE PASSAGEM:
lista_filtrada = []

for df in resultados:
    # Filtrando as linhas onde a coluna 'distancia_acumulada' é menor que tamnho do trecho
    df_filtrado = df.query('distancia_acumulada < 2583.53007784341')
    lista_filtrada.append(df_filtrado)

In [ ]:
    ''' AGORA IREMOS FILTRAR DESSA LISTA AQUELES PERFIS QUE TIVEREM MAIS DE 5 LINHAS, OU SEJA, SOMENTE OS QUE RELAMENTE ESTIVEREM NO SENTIDO TERÃO MAIS REGISTROS E OQUE NÃO TIVEREM VÃO TER POUCAS LINHAS APOS A APLICAÇÃO DO FILTRO ACIMA!  '''

In [ ]:
lista_final = []

for df in lista_filtrada:
    if len(df) > 8:
        lista_final.append(df)

In [ ]:
len(lista_final) 

In [ ]:
lista_final[0]

In [12]:
''' Ida quando se compara a distancia com a primeira parada '''

' Ida quando se compara a distancia com a primeira parada '

In [13]:
''' Volta quando os filtrados são os que possuem distancia em relação a ultima estação (ultima sentido centro) '''

' Volta quando os filtrados são os que possuem distancia em relação a ultima estação (ultima sentido centro) '

# 7.0 Base com perfis discretizados e analises iniciais
    ''' OBS: A PARTIR DESSE TOPICO SERÃO FEITAS AS ANALISES!  ''' 

## 7.1 Adicionando linha dos veiculos nos dados de perfis

In [ ]:
# Lista para armazenar os DataFrames mesclados
lista_mesclada = []

for df in lista_final:
    # Realizando o merge com base na coluna 'id_veiculo'
    df_mesclado = pd.merge(df, validações_linhas_hora[['linha', 'nome_linha', 'id_veiculo']],  on='id_veiculo')
    lista_mesclada.append(df_mesclado)

In [ ]:
lista_mesclada[2]

In [14]:
''' AGORA A BASE TEM A LINHA QUE O CADA VEICULO FAZ PARTE! '''

' AGORA A BASE TEM A LINHA QUE O CADA VEICULO FAZ PARTE! '

## 7.2 Discretizando cada perfil 
''' Assumindo que a componente de velocidade a cada trecho discretizado seja proporcional a velocidade no trecho completo '''

In [ ]:
# Discretizando a cada 5 metros
# Lista para armazenar os DataFrames resultantes
lista_velocidades_discretizadas = []

for df in lista_mesclada: 
    # Inicializando a distância acumulada
    distancia_acumulada = 0
    
    # Obtendo o tamanho total do trecho
    tamanho_total_trecho = df['distancia_acumulada'].max()
    
    # Criando um DataFrame para armazenar as velocidades discretizadas
    velocidades_discretizadas = pd.DataFrame(columns=['distancia', f'velocidade_{df.iloc[0]["linha"]}_{df.iloc[0]["id_veiculo"]}'])
    
    linha_atual = df.iloc[0]['linha']  # Obtendo a informação da linha do primeiro DataFrame
    id_veiculo_atual = df.iloc[0]['id_veiculo']  # Obtendo a informação do id_veiculo do primeiro DataFrame
    
    # Iterando sobre os trechos do DataFrame
    for index, row in df.iterrows():
        distancia_trecho_atual = row['distancia']
        velocidade_trecho_atual = row['velocidade']
    
        # Adicionando as distâncias discretizadas ao DataFrame de velocidades discretizadas
        for distancia in range(0, int(distancia_trecho_atual) + 1, 5):
            if distancia_acumulada > tamanho_total_trecho:
                break
            velocidades_discretizadas = velocidades_discretizadas.append({'distancia': distancia_acumulada, 
                                                                          f'velocidade_{linha_atual}_{id_veiculo_atual}': velocidade_trecho_atual}, 
                                                                         ignore_index=True)
            distancia_acumulada += 5
    
    lista_velocidades_discretizadas.append(velocidades_discretizadas)

In [ ]:
lista_velocidades_discretizadas[4]

In [ ]:
# Fator de conversão de m/s para km/h
fator_conversao = 3.6

for df in lista_velocidades_discretizadas:
    # Iterando sobre as colunas do DataFrame
    for coluna in df.columns:
        # Verificando se o nome da coluna contém a palavra "velocidade"
        if 'velocidade' in coluna:
            # Convertendo a coluna de velocidade de m/s para km/h
            df[coluna] *= fator_conversao

## 7.3 Visualização e resumo dos perfis no trecho e horario especifico

In [ ]:
# Lista para armazenar os traces de velocidade de cada DataFrame
traces = []

# Iterando sobre cada DataFrame da lista
for idx, df in enumerate(lista_velocidades_discretizadas):
    # Extraindo informações da coluna de velocidade
    info_velocidade = df.columns[-1].split('_')
    linha_atual = info_velocidade[1]  # Obtendo o número da linha
    id_veiculo_atual = info_velocidade[2]  # Obtendo o ID do veículo
    
    # Criando um trace para a velocidade de cada DataFrame
    trace = go.Scatter(
        x=df['distancia'],
        y=df.iloc[:, -1],  # Utilizando a última coluna do DataFrame como velocidade
        mode='lines',
        name=f'Velocidade - Linha {linha_atual} - ID {id_veiculo_atual}',  # Nomeando o trace com informações dinâmicas
        hoverinfo='x+y+name',  # Informações mostradas ao passar o mouse sobre o gráfico
    )
    traces.append(trace)

# Preenchendo o valor nulo na primeira linha da coluna "distancia_acumulada" com zero
paradas_interesse['distancia_acumulada_paradas'].iloc[0] = 0

# Criando um trace para as paradas de interesse
trace_paradas = go.Scatter(
    x=paradas_interesse['distancia_acumulada_paradas'],  # Coordenadas x
    y=[0] * len(paradas_interesse),  # Coordenadas y (todos são 0)
    mode='markers',  # Modo de plotagem de marcadores
    marker=dict(
        symbol='triangle-up',  # Formato do marcador (triângulo para cima)
        size=10,  # Tamanho do marcador
        color='black',  # Cor do marcador
    ),
    hovertext=paradas_interesse['stop_name'],  # Texto de dica ao passar o mouse sobre os marcadores
    hoverinfo='text',  # Informações mostradas ao passar o mouse
    name='Estações BRT'  # Nome do trace
)

# Adicionando o trace das paradas de interesse à lista de traces
traces.append(trace_paradas)

# Criando o layout do gráfico
layout = go.Layout(
    title='Velocidades (veículos das linhas no trecho e horário de interesse)',
    xaxis=dict(title='Distância (m)'),
    yaxis=dict(title='Velocidade (km/h)'),
    hovermode='closest'
)

# Criando a figura com os traces e o layout
fig = go.Figure(data=traces, layout=layout)

# Exibindo o gráfico
fig.show()

In [16]:
''' SEGUE DESCRIÇÃO ESTATISTICA DE CADA PERFIL: '''

' SEGUE DESCRIÇÃO ESTATISTICA DE CADA PERFIL: '

In [ ]:
for idx, df in enumerate(lista_velocidades_discretizadas):
    # Obtendo o nome da coluna de velocidade
    nome_coluna_velocidade = df.columns[-1]
    
    # Calculando o resumo estatístico apenas para a coluna de velocidade
    resumo_velocidade = df[nome_coluna_velocidade].describe()
    
    # Imprimindo o resumo estatístico apenas para a coluna de velocidade
    print(f"Resumo Estatístico do perfil de velocidade ({nome_coluna_velocidade}):")
    print(resumo_velocidade)
    print()  # Adiciona uma linha em branco entre os resumos estatísticos

In [15]:
''' OBSERVAR A MÉDIA DE VELOCIDADES E DISPERSÃO DOS VALORES'''

' OBSERVAR A MÉDIA DE VELOCIDADES E DISPERSÃO DOS VALORES'

## 7.4 Exportação
''' COM DADOS DE PERFIS VEICULARES DE DIFERENTES LINHAS NO MESMO INTERVALO, BASTA EXPORTALOS E GERAR PARA DEMAIS HORARIOS DE INTERSSE, DIAS E PROCEGUIR COM AS DEMAIS ANALISES! ''' 

In [ ]:
# Definindo o nome do arquivo Excel
nome_arquivo_excel = "dados__perfis_velocidades_dia01_17_18hs.xlsx"

# Criando um DataFrame vazio para armazenar os dados finais
df_final = pd.DataFrame(columns=['distancia'])

# Iterando sobre cada DataFrame na lista
for idx, df in enumerate(lista_velocidades_discretizadas):
    # Adicionando a coluna de distância ao DataFrame final
    df_final['distancia'] = df['distancia']
    
    # Obtendo o nome da coluna de velocidade
    nome_coluna_velocidade = df.columns[-1]
    
    # Adicionando as colunas de velocidade discretizada ao DataFrame final
    df_final[nome_coluna_velocidade] = df.iloc[:, -1]

# Exportando o DataFrame final para o arquivo Excel
df_final.to_excel(nome_arquivo_excel, index=False)

# --------------------------------------------------------------------------------------